In [226]:
# Para llevar el registro
!pip install -q comet_ml
import comet_ml
comet_ml.init(project_name = 'FineTunning')

In [227]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, Concatenate, PReLU
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Adadelta
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.regularizers import l2

In [228]:
# Se crea el experimento en Comet
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging = True,
    auto_histogram_gradient_logging = True,
    auto_histogram_activation_logging = True,
    log_code = True
)
experiment.set_name('Test 25')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gurumasterahb/finetunning/4a5deb7710274639b7ae768b983f83d2



In [229]:
# Parámetros
i_width = int(178 * 1.2)
i_height = int(218 * 1.2)
input_shape = (i_width, i_height, 3)

parameters = {
    'num_class' : 2,
    'epochs' : 200,
    'batch_size' : 16,
    'learning_rate' : 0.01,
    'loss' : 'binary_crossentropy',
    'optimizer' : 'RMSprop'}

experiment.log_parameters(parameters)

In [230]:
# Loading images
train_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/train'
val_dir = '/kaggle/input/data-finetunning/Norma_align/Norma_align/val'

# Image Generator
gen_train = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,  # % de desplazamiento horizontal
    height_shift_range = 0.15, # % de desplazamiento vertical
    brightness_range = [0.25, 1.25], # Porcentaje de brillo en el rango
    zoom_range = 0.3, # Zoom de hasta 30%
    channel_shift_range = 20, # Cada pixel de cada canal varía su intensidad en una vecindad de 20 unidades
    fill_mode = 'nearest', # Después de desplazamientos que dejan la imágen 'incompleta' se rellena con el valor de los pixeles más cercanos
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
train = gen_train.flow_from_directory(
    train_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

gen_val = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    brightness_range = [0.25, 1.25],
    zoom_range = 0.3,
    channel_shift_range = 20,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255.)
val = gen_val.flow_from_directory(
    val_dir,
    target_size = (i_width, i_height),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = parameters['batch_size'],
    shuffle = True)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [231]:
# Loading model
path_load = os.path.join('/kaggle/input/data-finetunning/best_val_loss.h5')
model_loaded = tf.keras.models.load_model(path_load)

model_0 = Sequential()

for layer in model_loaded.layers[:15]:
    layer.trainable = False
    model_0.add(layer)

# model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.01)))
model_0.add(Dense(512, activation = 'PReLU'))
model_0.add(Dense(256, activation = 'PReLU'))
# model_0.add(Dropout(0.12))
model_0.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.05)))
model_0.add(Dropout(0.5))
model_0.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.02)))
model_0.add(Dense(1, activation = 'sigmoid'))
    
# model_0.summary()

In [232]:
# Checkpoint para el mejor modelo (val_accuracy)
filepath = 'best_val_loss.h5'
checkpoint = ModelCheckpoint(
    filepath,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min')

In [233]:
# Compilación y entrenamiento
model_0.compile(
    loss = parameters['loss'],
    optimizer = RMSprop(learning_rate = parameters['learning_rate']),
    metrics = ['accuracy'])

model_0.fit(
    train,
    epochs = parameters['epochs'],
    verbose = 1,
    validation_data = val,
    callbacks = [checkpoint])

Epoch 1/200
8/8 [==============================] - ETA: 0s - loss: 4.3000 - accuracy: 0.5083
Epoch 1: val_loss improved from inf to 1.58380, saving model to best_val_loss.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 6s 485ms/step - loss: 4.3000 - accuracy: 0.5083 - val_loss: 1.5838 - val_accuracy: 0.5000
Epoch 2/200
8/8 [==============================] - ETA: 0s - loss: 1.3209 - accuracy: 0.4500
Epoch 2: val_loss improved from 1.58380 to 1.11985, saving model to best_val_loss.h5
8/8 [==============================] - 3s 470ms/step - loss: 1.3209 - accuracy: 0.4500 - val_loss: 1.1198 - val_accuracy: 0.5250
Epoch 3/200
8/8 [==============================] - ETA: 0s - loss: 1.0516 - accuracy: 0.5833
Epoch 3: val_loss did not improve from 1.11985
8/8 [==============================] - 4s 486ms/step - loss: 1.0516 - accuracy: 0.5833 - val_loss: 2.0718 - val_accuracy: 0.7000
Epoch 4/200
8/8 [==============================] - ETA: 0s - loss: 0.9090 - accuracy: 0.6417
Epoch 4: val_loss did not improve from 1.11985
8/8 [==============================] - 4s 507ms/step - loss: 0.9090 - accuracy: 0.6417 - val_loss: 1.4545 - val_accuracy: 0.6500
Epoch 5/200
8/8 [=========

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 4s 470ms/step - loss: 0.7243 - accuracy: 0.6833 - val_loss: 0.6477 - val_accuracy: 0.6750
Epoch 15/200
8/8 [==============================] - ETA: 0s - loss: 0.6386 - accuracy: 0.7583
Epoch 15: val_loss did not improve from 0.64774
8/8 [==============================] - 4s 477ms/step - loss: 0.6386 - accuracy: 0.7583 - val_loss: 0.9047 - val_accuracy: 0.5500
Epoch 16/200
8/8 [==============================] - ETA: 0s - loss: 0.6216 - accuracy: 0.7667
Epoch 16: val_loss did not improve from 0.64774
8/8 [==============================] - 4s 457ms/step - loss: 0.6216 - accuracy: 0.7667 - val_loss: 0.7330 - val_accuracy: 0.6750
Epoch 17/200
8/8 [==============================] - ETA: 0s - loss: 0.5849 - accuracy: 0.7917
Epoch 17: val_loss did not improve from 0.64774
8/8 [==============================] - 3s 434ms/step - loss: 0.5849 - accuracy: 0.7917 - val_loss: 0.6980 - val_accuracy: 0.6750
Epoch 18/200
8/8 [==============================] - ETA: 0

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 3s 422ms/step - loss: 0.6415 - accuracy: 0.7417 - val_loss: 0.6089 - val_accuracy: 0.6750
Epoch 24/200
8/8 [==============================] - ETA: 0s - loss: 0.6402 - accuracy: 0.7083
Epoch 24: val_loss did not improve from 0.60889
8/8 [==============================] - 3s 420ms/step - loss: 0.6402 - accuracy: 0.7083 - val_loss: 0.7929 - val_accuracy: 0.6000
Epoch 25/200
8/8 [==============================] - ETA: 0s - loss: 0.5913 - accuracy: 0.7583
Epoch 25: val_loss did not improve from 0.60889
8/8 [==============================] - 3s 422ms/step - loss: 0.5913 - accuracy: 0.7583 - val_loss: 1.0111 - val_accuracy: 0.6000
Epoch 26/200
8/8 [==============================] - ETA: 0s - loss: 0.7004 - accuracy: 0.7417
Epoch 26: val_loss improved from 0.60889 to 0.60838, saving model to best_val_loss.h5
8/8 [==============================] - 3s 429ms/step - loss: 0.7004 - accuracy: 0.7417 - val_loss: 0.6084 - val_accuracy: 0.7500
Epoch 27/200
8/8 [==

In [234]:
experiment.log_model("Facial_r", filepath)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/finetunning/4a5deb7710274639b7ae768b983f83d2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [200]                : (0.44999998807907104, 0.8916666507720947)
COMET INFO:     batch_accuracy [200]          : (0.25, 1.0)
COMET INFO:     batch_loss [200]              : (0.20630820095539093, 10.995519638061523)
COMET INFO:     epoch_duration [200]          : (2.7202139430009993, 5.967596359001618)
COMET INFO:     loss [200]                    : (0.3647645115852356, 4.299959182739258)
COMET INFO:     val_accuracy [200]            : (0.5, 0.875)
COMET INFO:     val_loss [200]                : (0.3871200